In [1]:
import numpy as np
import os

In [2]:

#-------------------------------------------------------------
# FUNCTION: ymd2dn
# Convert from year, month, day to day number of the year.
#-------------------------------------------------------------
def ymd2dn(year, month, day):
    """
    Convert from year, month, and day to the day number of the year.
    
    Parameters:
    year : int or list of ints
        The year (e.g., 1988).
    month : int or list of ints
        The month number (e.g., 11 for November).
    day : int or list of ints
        The day of the month.
    
    Returns:
    day_num : int
        The day number of the year.
    """
    # Days before the start of each month (non-leap year)
    idays = [0, 31, 59, 90, 120, 151, 181, 212, 243, 273, 304, 334, 366]

    # Check if it's a leap year
    leap_year = ((year % 4 == 0) and (year % 100 != 0)) or (year % 400 == 0)

    # Adjust for leap year if month is >= March
    leap_correction = leap_year and month >= 3

    day_num = day + idays[month - 1] + leap_correction
    return day_num

#-------------------------------------------------------------
# FUNCTION: interpol_rk
# Fast interpolation of data with possible bad values.
#-------------------------------------------------------------
def interpol_rk(x, y, xx, yrange=None):
    """
    Fast interpolation of data, with handling of bad values (out of range).
    
    Parameters:
    x : array-like
        Input x values.
    y : array-like
        Input y values.
    xx : array-like
        Desired x values for interpolation.
    yrange : tuple, optional
        Tuple defining the range (min, max) of y values to consider valid.
    
    Returns:
    yy : array
        Interpolated y values at positions xx.
    """
    x2, y2 = np.copy(x), np.copy(y)
    
    # Remove values out of the specified yrange
    if yrange is not None:
        valid = (y > min(yrange)) & (y < max(yrange))
        if np.sum(valid) > 0:
            x2, y2 = x[valid], y[valid]
    
    # Perform linear interpolation
    yi = np.interp(xx, np.arange(len(y2)), x2)
    yy = np.interp(yi, np.arange(len(y2)), y2)
    
    return yy

#-------------------------------------------------------------
# PROCEDURE: nict_sw
# Read solar wind data and interpolate over missing data.
#-------------------------------------------------------------
def nict_sw(year, month, day):
    """
    Read solar wind data from file, process, and save interpolated data.
    
    Parameters:
    year : int
        Year of the data.
    month : int
        Month of the data.
    day : int
        Day of the data.
    """
    # Constants
    v0 = 5.32e4
    b0 = 10.0e-9
    n0 = 10.0

    # Construct file paths
    dir_path = 'D:/nict05/'
    os.chdir(dir_path)

    yyyymmdd = f"{year:04d}{month:02d}{day:02d}"
    doy = ymd2dn(year, month, day)
    
    doys = np.sin(doy / 365.0 * 2 * np.pi)
    doyc = np.cos(doy / 365.0 * 2 * np.pi)

    # Initialize arrays
    tt = np.zeros(1445)
    ts = np.zeros(1445)
    tc = np.zeros(1445)
    nn = np.zeros(1445)
    vv = np.zeros(1445)
    by = np.zeros(1445)
    bz = np.zeros(1445)

    # Read file and process data
    fn = yyyymmdd + 'sw.txta'
    with open(fn, 'r') as f:
        nl = 0
        for line in f:
            d = np.array([float(i) for i in line.split()])
            t = d[1] * 60 + d[2]
            n = d[3] * n0
            v = d[4] * v0
            y = d[5] * b0
            z = d[6] * b0

            tt[nl] = t
            ts[nl] = np.sin(t / 1440.0 * 2 * np.pi)
            tc[nl] = np.cos(t / 1440.0 * 2 * np.pi)
            nn[nl] = n
            vv[nl] = v
            by[nl] = y
            bz[nl] = z
            nl += 1

    # Truncate arrays to valid data length
    tt = tt[:nl]
    ts = ts[:nl]
    tc = tc[:nl]
    nn = nn[:nl]
    vv = vv[:nl]
    by = by[:nl]
    bz = bz[:nl]

    # Interpolate data
    t2 = np.arange(1440)
    n2 = interpol_rk(tt, nn, t2)
    v2 = interpol_rk(tt, vv, t2) * 1e-3
    y2 = interpol_rk(tt, by, t2) * 1e9
    z2 = interpol_rk(tt, bz, t2) * 1e9

    ds2 = np.full(1440, doys)
    dc2 = np.full(1440, doyc)
    ts2 = interpol_rk(tt, ts, t2)
    tc2 = interpol_rk(tt, tc, t2)

    # Save the interpolated data
    np.savez(dir_path + yyyymmdd + '_ssw.npz', ds2=ds2, dc2=dc2, ts2=ts2, tc2=tc2, n2=n2, v2=v2, y2=y2, z2=z2)
    
# Example usage
nict_sw(2021, 7, 29)


FileNotFoundError: [Errno 2] No such file or directory: 'D:/nict05/'